# Genetic Algorithm

This notebook showcases how to use the built-in Genetic Algorithm (GA)

In [49]:
using Statistics
using EvoLP
using OrderedCollections

For this example we will use the **Rosenbrock** function:

In [50]:
@doc rosenbrock

```
rosenbrock(x; a=1, b=5)
```

**Rosenbrock** benchmark function. With `a=1` and `b=5`, minimum is at `f([a, a^2]) = 0`

``
f(x) = (a - x_1)^2 + b(x_2 - x_1^2)^2
``


In a GA, we use vectors as _individuals_.

Let's start creating the population. For that, we can use a generator. Let's use the normal generator:

In [51]:
@doc normal_rand_vector_pop

```
normal_rand_vector_pop(n, μ, Σ; rng=Random.GLOBAL_RNG)
```

Generate a population of `n` vector individuals using a normal distribution with means `μ` and covariance `Σ`.

`μ` expects a vector of length *l* (i.e. length of an individual) while `Σ` expects an *l x l* matrix of covariances.

# Examples

```julia
julia> normal_rand_vector_pop(3, [0, 0], [1 0; 0 1])
3-element Vector{Vector{Float64}}:
 [-0.15290525182234904, 0.8715880371871617]
 [-1.1283800329864322, -0.9256584563613383]
 [-0.5384758126777555, -0.8141702145510666]
```


In [52]:
pop_size = 50
population = normal_rand_vector_pop(pop_size, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Vector{Float64}}:
 [0.6066134412734522, -0.37262517254346783]
 [1.1578326727914892, -0.593229583706778]
 [0.4146002932358113, -0.4846849429791206]

In a GA, we have _selection_, _crossover_ and _mutation_.

We can easily set up these operators using the built-ins provided by EvoLP.

Let's use rank based selection and interpolation crossover with 0.5 as the scaling factor:

In [53]:
@doc InterpolationCrossover

Interpolation crossover with scaling parameter `λ`.


In [54]:
S = RankBasedSelectionGenerational()
C = InterpolationCrossover(0.5)

InterpolationCrossover(0.5)

For mutation, we can use a Gaussian approach:

In [55]:
@doc GaussianMutation

Gaussian mutation with standard deviation `σ`, which should be a real number.


In [56]:
M = GaussianMutation(0.5)

GaussianMutation(0.5)

We can use the `Logbook` to record statistics about our run:

In [57]:
statnames = ["mean_eval", "max_f", "min_f", "median_f"]
fns = [mean, maximum, minimum, median]
thedict = LittleDict(statnames, fns)
thelogger = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("mean_eval" => Statistics.mean, "max_f" => maximum, "min_f" => minimum, "median_f" => Statistics.median), NamedTuple{(:mean_eval, :max_f, :min_f, :median_f)}[])

And now we're ready to use the `GA` built-in algorithm:

In [58]:
@doc GA

```
GA(f, pop, k_max, S, C, M)
GA(logbook::Logbook, f, pop, k_max, S, C, M)
```

Generational Genetic Algorithm.

## Arguments

  * `f::Function`: objective function to **minimise**.
  * `pop::AbstractVector`: population—a list of vector individuals.
  * `k_max::Integer`: number of iterations.
  * `S::SelectionMethod`: one of the available [`SelectionMethod`](@ref).
  * `C::CrossoverMethod`: one of the available [`CrossoverMethod`](@ref).
  * `M::MutationMethod`: one of the available [`MutationMethod`](@ref).

Returns a [`Result`](@ref).


In [59]:
result = GA(thelogger, rosenbrock, population, 500, S, C, M);

The output was suppressed so that we can analyse each part of the result separately:

In [60]:
@show optimum(result)

@show optimizer(result)

@show f_calls(result)

@show thelogger.records[end]

optimum(result) = 0.0015029528354023858
optimizer(result) = [1.0367119356341026, 1.0803427525882299]
f_calls(result) = 50050
thelogger.records[end] = (mean_eval = 3.7839504926952294, max_f = 22.281919411164413, min_f = 0.0015029528354023858, median_f = 2.429775485243721)


(mean_eval = 3.7839504926952294, max_f = 22.281919411164413, min_f = 0.0015029528354023858, median_f = 2.429775485243721)

In [61]:
using DataFrames

In [62]:
DataFrame(thelogger.records)

Row,mean_eval,max_f,min_f,median_f
,Float64,Float64,Float64,Float64
1,9.89648,134.066,0.149169,3.6715
2,5.73033,40.1663,0.134373,2.94368
3,4.65904,18.9003,0.181144,3.22389
4,4.34055,52.1745,0.0813748,2.14014
5,5.1239,42.4315,0.119411,2.83591
6,3.70733,37.285,0.0413168,2.17732
7,4.84669,66.9539,0.00445092,2.52908
8,3.87748,26.8901,0.345194,1.75084
9,6.35524,69.8636,0.172651,2.32515
